In [2]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
import librosa

# Use processor instead of tokenizer for audio
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# Load your audio
audio_path = "audio/pi.mp3"
audio, sr = librosa.load(audio_path, sr=16000)

# Process audio with processor (not tokenizer)
inputs = processor(audio, sampling_rate=16000, return_tensors="pt")

# Get predictions
with torch.no_grad():
    logits = model(**inputs).logits
    predicted_ids = torch.argmax(logits, dim=-1)

# Decode to text
transcription = processor.decode(predicted_ids[0])
print("Transcription:", transcription)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[src/libmpg123/id3.c:process_comment():587] error: No comment text / valid description?


Transcription: HETHERON I RYGYOUNG AND THIS IS THE THE MOST TACING YEAR FOR PARTI INMENTATION TO TIMES OUR SYSTEM IS RESPONSIBLE FOR PERRLYBE CHECKING THE THIS LEVIL OR BABY AND OLORTIN TE CHARITY WERE COLORTING THE PERSONAL ANGES BEAR WILES COMMEATION INGAVOF ANYDANGEN A SO BERATUALLY ASTRATLERYLY CAVOLITION OF THIS LEVIL INSTEAD BE USED TO SLICHES THIS IS GONLY REPLACE BY ACTUAL N FOR IT ON AND AND ANOOL ANOLO IMPORT E IN THE P I FIE A SO WE HAVE TOST ME JUST HERE A AR OUT O COMPONDS TAR AED AND BAZZAR AND WE ALSO HAVE ANOTHER SAGE JUST FOR THIS IS JUST WOKING ELEQICALY TO STOGBLE BETWEEN SOUND AND SILENT MOD AND TUS TALK ABOUT THE RALL OF TIMOR IN OUR P I WE HAVE TWO TIMES FOR THIS PEI ONE FOR PELYLY CHECKING BY CAKING I MEAN CHAKING THE IMPLS A O OUR SWICHES CHAKING THERE ER THIS LEVEL AND ANOTHER HIMOL IS FO ADJUSTING THE FREQONCY OR HARPE COMPONDS EITHER  LED OR BOT OTHE LETSERIN IN ACTION CANRONTLY THENYE NOL IS SAID TO ZEAO AND WE ARE IN A SONE MD LET SEE WHAT HAPPEN SPENDE DIN